# Code for generating and running PH of noisy networks - dev

In [1]:
# Need to load in all the network generation functions here?


In [2]:
# Load packages

using Pkg
using Statistics
using Plots
using LinearAlgebra
using Distances
using Eirene
using StatsBase
using Random
using Distributions
using JLD
using MAT

include("graph_models.jl")
include("helper_functions.jl")


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17958264158906669996\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-12609990072733088897\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17958264158906669996\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

betticurveFromBarcode (generic function with 1 method)

## Choose weighed network and generate replicates

--------------------------------
## CHANGE PARAMETERS HERE
---------------------------------

In [3]:
## Pick from one of the graph models and generate out to THRESH edge density

nReps = 50
rho = 0.4      # Threshold at edge density = rho
thresh = "04"
nNodes = 70
save_figures = 1  # Boolean to save figure pdfs
save_data = 1    # Boolean to save data  
date_string = "082420"

# for geometricConf
p = 0.01
scale_factor = 100

# for RG
dims = 3

# for discreteUniformConf
a = 0
b = 1000

# for cosineGeometric
dims = 3


# Graph model name one of "geometricConf", "IID" , "RG", "discreteUniformConf", "cosineGeometric", "RL", "assoc"
# "disassort", "coreperiph", "DP"
graph_model_names = ["geometricConf", "IID" , "RG", "discreteUniformConf", "cosineGeometric", "RL", "assoc","disassort", "coreperiph", "DP"]

10-element Array{String,1}:
 "geometricConf"      
 "IID"                
 "RG"                 
 "discreteUniformConf"
 "cosineGeometric"    
 "RL"                 
 "assoc"              
 "disassort"          
 "coreperiph"         
 "DP"                 

In [4]:
myreaddir = "/home/jovyan/processed_data/70nodes/thresh_$(thresh)"
eirene_output_files = filter(x->occursin("graphs.jld",x), readdir(myreaddir))

nModels = length(eirene_output_files)
model_names = [split(eirene_output_files[i],"_")[1] for i=1:nModels]



11-element Array{SubString{String},1}:
 "DP"                 
 "IID"                
 "RG"                 
 "RL"                 
 "assoc"              
 "coreperiph"         
 "cosineGeometric"    
 "disassort"          
 "disassort"          
 "discreteUniformConf"
 "geometricConf"      

In [5]:



for (i,file) in enumerate(eirene_output_files)
    
    println(model_names[i])
    
    
    # load in data
    array_dict = load("$(myreaddir)/$(file)")
    weighted_graph_array_iidNoise = array_dict["weighted_graph_array_iidNoise"]
    # When making the matrices, we added 1 to every real value and added noise in [0,1]. So every value <1 should be noise
    noise_independent_array = deepcopy(weighted_graph_array_iidNoise)
    noise_independent_array[noise_independent_array.>1].= 0

    maxDim = 3
    nEdges = binomial(nNodes,2)
    barcode_array = Array{Array{Float64}}(undef,nReps,maxDim)
    betticurve_array = zeros(nEdges,maxDim,nReps)
    beginTime = time()



    for rep = 1:nReps
        
        G_i_rand = noise_independent_array[:,:,rep]
        
        # Now create the ranked matrix for running eirene -- it will save us many headaches later.
        edge_list_ranks = denserank([G_i_rand...], rev = true)   # so highest edge weight gets assigned 1
        G_i_rand_ord = reshape(edge_list_ranks,(nNodes,nNodes))
        G_i_rand_ord[diagind(G_i_rand_ord)] .= 0

        C = Eirene.eirene(G_i_rand_ord,model = "vr", maxdim = maxDim, record = "none")
        println("finished running eirene")
        for k in collect(1:maxDim)

            barcode_array[rep,k] = barcode(C,dim=k)

        end 

        #if rep%10 == 0
            printstyled("\nRun $(rep) finished.\n",color = :cyan)
        #end

        C=0
        endTime = time() - beginTime
        printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

    end



    endTime = time() - beginTime
    printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

    if save_data == 1
        save("../processed_data/$(model_names[i])_$(date_string)_noiseIndependent_eirene_output.jld",
            "barcode_array", barcode_array,
            "betticurve_array", betticurve_array)
    end


    printstyled("\nDone :)", color = :blue)


    beginTime = time()
    for rep = 1:nReps

        for k in collect(1:maxDim)

            betti_curve_i = betticurveFromBarcode(barcode_array[rep,k],nEdges)
            betticurve_array[:,k,rep] = betti_curve_i[1,:]
            

        end 


    end


    ## Plot average betti curves
    ### UPDATE FOR DIM 5

    if save_figures == 1
        betticurve_avg = dropdims(mean(betticurve_array,dims=3), dims = 3)
        betticurve_std = dropdims(std(betticurve_array,dims=3), dims = 3)

        x_axis = collect(1:nEdges) ./nEdges
        p1a = plot(x_axis,betticurve_avg[:,1],linewidth = 1.5, ribbon = betticurve_std[:,1], label = "Betti_1")
        for d = 2:maxDim
            plot!(x_axis,betticurve_avg[:,d],linewidth = 1.5, ribbon = betticurve_std[:,d], label = "Betti_$(d)")
        end

        title!("$(model_names[i])")
        ylabel!("Betti_k")
        xlabel!("Edge Density")

        plot(p1a,layout = (1,1))


        savefig("../figures/$(model_names[i])_$(date_string)_noiseIndependent_bettis.pdf")
    end
end



DP
finished running eirene

Run 1 finished.
Finished computing persistent homology, elapsed time 10.587347030639648 secondsfinished running eirene

Run 2 finished.
Finished computing persistent homology, elapsed time 10.836684942245483 secondsfinished running eirene

Run 3 finished.
Finished computing persistent homology, elapsed time 11.235337972640991 secondsfinished running eirene

Run 4 finished.
Finished computing persistent homology, elapsed time 11.40135908126831 secondsfinished running eirene

Run 5 finished.
Finished computing persistent homology, elapsed time 11.717280864715576 secondsfinished running eirene

Run 6 finished.
Finished computing persistent homology, elapsed time 11.946156024932861 secondsfinished running eirene

Run 7 finished.
Finished computing persistent homology, elapsed time 12.278707027435303 secondsfinished running eirene

Run 8 finished.
Finished computing persistent homology, elapsed time 12.611468076705933 secondsfinished running eirene

Run 9 finishe

Finished computing persistent homology, elapsed time 13.15313720703125 secondsfinished running eirene

Run 9 finished.
Finished computing persistent homology, elapsed time 14.904968023300171 secondsfinished running eirene

Run 10 finished.
Finished computing persistent homology, elapsed time 16.417713165283203 secondsfinished running eirene

Run 11 finished.
Finished computing persistent homology, elapsed time 17.96686315536499 secondsfinished running eirene

Run 12 finished.
Finished computing persistent homology, elapsed time 19.67317008972168 secondsfinished running eirene

Run 13 finished.
Finished computing persistent homology, elapsed time 21.112577199935913 secondsfinished running eirene

Run 14 finished.
Finished computing persistent homology, elapsed time 22.722487211227417 secondsfinished running eirene

Run 15 finished.
Finished computing persistent homology, elapsed time 24.30574917793274 secondsfinished running eirene

Run 16 finished.
Finished computing persistent homolog

Finished computing persistent homology, elapsed time 22.2695369720459 secondsfinished running eirene

Run 17 finished.
Finished computing persistent homology, elapsed time 23.70329999923706 secondsfinished running eirene

Run 18 finished.
Finished computing persistent homology, elapsed time 25.063822031021118 secondsfinished running eirene

Run 19 finished.
Finished computing persistent homology, elapsed time 26.419918060302734 secondsfinished running eirene

Run 20 finished.
Finished computing persistent homology, elapsed time 27.897303104400635 secondsfinished running eirene

Run 21 finished.
Finished computing persistent homology, elapsed time 29.200464963912964 secondsfinished running eirene

Run 22 finished.
Finished computing persistent homology, elapsed time 30.698116064071655 secondsfinished running eirene

Run 23 finished.
Finished computing persistent homology, elapsed time 32.02081894874573 secondsfinished running eirene

Run 24 finished.
Finished computing persistent homolo

Finished computing persistent homology, elapsed time 31.305863857269287 secondsfinished running eirene

Run 25 finished.
Finished computing persistent homology, elapsed time 32.69342398643494 secondsfinished running eirene

Run 26 finished.
Finished computing persistent homology, elapsed time 33.9119029045105 secondsfinished running eirene

Run 27 finished.
Finished computing persistent homology, elapsed time 35.32344198226929 secondsfinished running eirene

Run 28 finished.
Finished computing persistent homology, elapsed time 36.519611835479736 secondsfinished running eirene

Run 29 finished.
Finished computing persistent homology, elapsed time 37.926440954208374 secondsfinished running eirene

Run 30 finished.
Finished computing persistent homology, elapsed time 39.119717836380005 secondsfinished running eirene

Run 31 finished.
Finished computing persistent homology, elapsed time 40.50870394706726 secondsfinished running eirene

Run 32 finished.
Finished computing persistent homolog

Finished computing persistent homology, elapsed time 59.94102597236633 secondsfinished running eirene

Run 33 finished.
Finished computing persistent homology, elapsed time 61.72020602226257 secondsfinished running eirene

Run 34 finished.
Finished computing persistent homology, elapsed time 63.3837571144104 secondsfinished running eirene

Run 35 finished.
Finished computing persistent homology, elapsed time 65.00646209716797 secondsfinished running eirene

Run 36 finished.
Finished computing persistent homology, elapsed time 66.94418001174927 secondsfinished running eirene

Run 37 finished.
Finished computing persistent homology, elapsed time 68.54003405570984 secondsfinished running eirene

Run 38 finished.
Finished computing persistent homology, elapsed time 70.09426212310791 secondsfinished running eirene

Run 39 finished.
Finished computing persistent homology, elapsed time 71.99052000045776 secondsfinished running eirene

Run 40 finished.
Finished computing persistent homology, e

Finished computing persistent homology, elapsed time 60.11138606071472 secondsfinished running eirene

Run 41 finished.
Finished computing persistent homology, elapsed time 61.80279898643494 secondsfinished running eirene

Run 42 finished.
Finished computing persistent homology, elapsed time 63.20464301109314 secondsfinished running eirene

Run 43 finished.
Finished computing persistent homology, elapsed time 64.92904996871948 secondsfinished running eirene

Run 44 finished.
Finished computing persistent homology, elapsed time 66.24295902252197 secondsfinished running eirene

Run 45 finished.
Finished computing persistent homology, elapsed time 67.96829104423523 secondsfinished running eirene

Run 46 finished.
Finished computing persistent homology, elapsed time 70.24489307403564 secondsfinished running eirene

Run 47 finished.
Finished computing persistent homology, elapsed time 71.70847010612488 secondsfinished running eirene

Run 48 finished.
Finished computing persistent homology, 

Finished computing persistent homology, elapsed time 64.29747486114502 secondsfinished running eirene

Run 48 finished.
Finished computing persistent homology, elapsed time 65.5445020198822 secondsfinished running eirene

Run 49 finished.
Finished computing persistent homology, elapsed time 67.14489388465881 secondsfinished running eirene

Run 50 finished.
Finished computing persistent homology, elapsed time 68.39065885543823 secondsFinished computing persistent homology, elapsed time 68.39068102836609 seconds
Done :)disassort
finished running eirene

Run 1 finished.
Finished computing persistent homology, elapsed time 1.500171184539795 secondsfinished running eirene

Run 2 finished.
Finished computing persistent homology, elapsed time 2.982196092605591 secondsfinished running eirene

Run 3 finished.
Finished computing persistent homology, elapsed time 4.572211980819702 secondsfinished running eirene

Run 4 finished.
Finished computing persistent homology, elapsed time 6.0611732006073 

Finished computing persistent homology, elapsed time 6.029260873794556 secondsfinished running eirene

Run 5 finished.
Finished computing persistent homology, elapsed time 7.473322868347168 secondsfinished running eirene

Run 6 finished.
Finished computing persistent homology, elapsed time 9.094600915908813 secondsfinished running eirene

Run 7 finished.
Finished computing persistent homology, elapsed time 10.625986814498901 secondsfinished running eirene

Run 8 finished.
Finished computing persistent homology, elapsed time 12.080015897750854 secondsfinished running eirene

Run 9 finished.
Finished computing persistent homology, elapsed time 13.868824005126953 secondsfinished running eirene

Run 10 finished.
Finished computing persistent homology, elapsed time 15.32303500175476 secondsfinished running eirene

Run 11 finished.
Finished computing persistent homology, elapsed time 16.861318826675415 secondsfinished running eirene

Run 12 finished.
Finished computing persistent homology, e

Finished computing persistent homology, elapsed time 2.1888160705566406 secondsfinished running eirene

Run 12 finished.
Finished computing persistent homology, elapsed time 2.310488224029541 secondsfinished running eirene

Run 13 finished.
Finished computing persistent homology, elapsed time 2.463778018951416 secondsfinished running eirene

Run 14 finished.
Finished computing persistent homology, elapsed time 2.7774150371551514 secondsfinished running eirene

Run 15 finished.
Finished computing persistent homology, elapsed time 2.8847320079803467 secondsfinished running eirene

Run 16 finished.
Finished computing persistent homology, elapsed time 3.0159990787506104 secondsfinished running eirene

Run 17 finished.
Finished computing persistent homology, elapsed time 3.1193912029266357 secondsfinished running eirene

Run 18 finished.
Finished computing persistent homology, elapsed time 3.2392220497131348 secondsfinished running eirene

Run 19 finished.
Finished computing persistent homo

Finished computing persistent homology, elapsed time 3.658466100692749 secondsfinished running eirene

Run 19 finished.
Finished computing persistent homology, elapsed time 3.885835886001587 secondsfinished running eirene

Run 20 finished.
Finished computing persistent homology, elapsed time 4.02128005027771 secondsfinished running eirene

Run 21 finished.
Finished computing persistent homology, elapsed time 4.136023044586182 secondsfinished running eirene

Run 22 finished.
Finished computing persistent homology, elapsed time 4.4634270668029785 secondsfinished running eirene

Run 23 finished.
Finished computing persistent homology, elapsed time 4.569117069244385 secondsfinished running eirene

Run 24 finished.
Finished computing persistent homology, elapsed time 4.6860740184783936 secondsfinished running eirene

Run 25 finished.
Finished computing persistent homology, elapsed time 4.804616928100586 secondsfinished running eirene

Run 26 finished.
Finished computing persistent homology,